In [ ]:
import torch
import numpy as np
import torch.nn as nn
import pandas as pd
import re
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score
import torchvision.transforms as transforms
from torchvision.io import read_image
from torch.utils.data import Dataset
from sklearn.model_selection import KFold
import random
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os
from torch.utils.data import DataLoader

Load the data

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip drive/My\ Drive/NN_201_Sarmanova/full_data.zip #full_data.zip located in NN_201_Sarmanova folder at google drive.

Archive:  drive/My Drive/NN_201_Sarmanova/full_data.zip
  inflating: full_data/1.csv         
  inflating: full_data/10.csv        
  inflating: full_data/100.csv       
  inflating: full_data/1000.csv      
  inflating: full_data/101.csv       
  inflating: full_data/102.csv       
  inflating: full_data/103.csv       
  inflating: full_data/104.csv       
  inflating: full_data/105.csv       
  inflating: full_data/106.csv       
  inflating: full_data/107.csv       
  inflating: full_data/108.csv       
  inflating: full_data/109.csv       
  inflating: full_data/11.csv        
  inflating: full_data/110.csv       
  inflating: full_data/111.csv       
  inflating: full_data/112.csv       
  inflating: full_data/113.csv       
  inflating: full_data/114.csv       
  inflating: full_data/115.csv       
  inflating: full_data/116.csv       
  inflating: full_data/117.csv       
  inflating: full_data/118.csv       
  inflating: full_data/119.csv       
  inflating: full_data/12.csv   

###Customized data class

In [ ]:
class CDS_2D_Dataset(Dataset):
    def __init__(self, annotations_file, spec_dir, transform=None, target_transform=None):
      '''annotations_file - the file which contains lables of the samples included in training/validation/test sets'''
      self.spec_labels = pd.read_csv(annotations_file, sep=',').iloc[:,1:] # labels (concentrations for 4 ions) for all the samples from annotation file (Y_ions.csv)
      self.spec_number = pd.read_csv(annotations_file, sep=',').iloc[:,0] # numbers for all the samples from annotation file (Y_ions.csv)
      self.spec_dir = spec_dir # folder where csv files are located
      self.transform = transform 
      self.target_transform = target_transform

    def __len__(self):
        return len(self.spec_labels)#length of the dataset

    def __getitem__(self, idx):
        label = self.spec_labels.iloc[idx] # get the label of the sample via sample's index
        sp = np.array(pd.read_csv(self.spec_dir + str(self.spec_number[idx])+'.csv').iloc[1:,1:-1], dtype='float32') # get the EEM of the sample via sample's index
        sp[sp<0]=0 # here we zero negative values of intensities
        spec = torch.from_numpy(sp).unsqueeze(0) # add dimension for channels of cnn
        
        if self.transform:
            spec = self.transform(spec)
        if self.target_transform:
            label = self.target_transform(label)
            
        return spec, torch.from_numpy(np.array(label, dtype='float32')) # return spectrum and corresponding labels

In [ ]:
#Calculate mean and std

def mean_std(train_dataloader):
    mean = 0.0
    for specs, _ in train_dataloader:
        batch_samples = specs.size(0) 
        specs = specs.view(batch_samples, specs.size(1), -1)
        mean += specs.mean(2).sum(0)
    mean = mean / len(train_dataloader.dataset)

    var = 0.0
    for specs, _ in train_dataloader:
        batch_samples = specs.size(0)
        specs = specs.view(batch_samples, specs.size(1), -1)
        var += ((specs - mean.unsqueeze(1))**2).sum([0,2])
    std = torch.sqrt(var / (len(train_dataloader.dataset)*500*41))

    return mean, std

Model

In [ ]:
#2D CNN class (basic)
class TwoDCNN(nn.Module):

    def __init__(self):
      
        super().__init__() # since Python 3.0
        

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)

        self.fc1 = nn.Linear(16 * 244 * 14, 160)
        self.fc2 = nn.Linear(160, 4)

    def forward(self, x):

        x = self.pool(F.relu(self.conv1(x)))
        x = F.relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
def reset_weights(m):
    '''
    Try resetting model weights to avoid weight leakage.
    '''
    for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            print(f'Reset trainable parameters of layer = {layer}')
            layer.reset_parameters()

In [ ]:
#Write the outputs of the network
def write_predictions(N, model_name, split_path,dataloader,dset):
    
    #load the model
    checkpoint = torch.load(split_path + 'model'+model_name+'.pth')
    N.load_state_dict(checkpoint['model_state_dict'])
    
    #load optimizer
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    last_best_epoch = checkpoint['epoch'] #the epoch with minimum loss function that occured during training
    loss = checkpoint['loss']
    N.eval()

    y_ae = np.zeros((1,4))
    y_ae_true = np.zeros((1,4))
    
    #write the outputs of the model
    for specs, labels in dataloader:
        outputs = N(specs) # get the outputs from the network
        outputs[outputs<0]=0 #we zero negative outputs as they are impossible for concentration values
        ae = outputs.detach().numpy()
        ae_true = labels.detach().numpy()
        #np.concatenate((y_ae, ae), axis=0)
        y_ae = np.concatenate((y_ae, ae), axis=0) # columns with networks's output for the dataloader
        y_ae_true = np.concatenate((y_ae_true, ae_true), axis=0) # columns with true output for the dataloader

    a = ['Cu','Ni','Cr','NO3']
    pd.DataFrame(y_ae).to_csv(split_path + 'Y_out_'+dset+'.csv',sep=',', header = a)
    pd.DataFrame(y_ae_true).to_csv(split_path + 'Y_true_'+dset+'.csv',sep=',', header = a)

In [ ]:
# calculate mae, rmse, r2 for dataloader
import math

def calculate_metrics(model, loader):
    running_mae = torch.zeros(1,4)
    running_mse = torch.zeros(1,4)
    y_sum = torch.zeros(1,4)
    num_examples = 0
    total_sum_squares = torch.zeros(1,4)
    target_mean = torch.zeros(1,4)

    for specs, targets in loader:
        y_sum += targets.sum(dim=0)
        num_examples += specs.shape[0]

    num_examples = 0
    for specs, targets in loader:
        preds = model(specs)
        preds[preds<0]=0

        num_examples += specs.shape[0] # batch size
        total_sum_squares += (torch.pow(preds - y_sum, 2)).sum(dim=0)

        error = torch.abs(preds - targets).sum(dim=0)
        squared_error = ((preds - targets)*(preds - targets)).sum(dim=0)
        running_mae += error
        running_mse += squared_error
  
  
    mae = (running_mae/num_examples).detach().numpy()
    rmse = (torch.sqrt(running_mse/num_examples)).detach().numpy()
    r2 = (1 - squared_error / total_sum_squares).detach().numpy()

    return mae, rmse, r2

#Augmetation

In [ ]:
# Wavelength drifting modelling

class Shift_by_px(object):
    def __init__(self, pixel=1):
        self.px = pixel
        
    def __call__(self, tensor):
    
        array = tensor.numpy()
        new = np.zeros(array.shape)
        dim = array.shape

        for st in range(0, dim[0]):
            for stol in range (0,dim[1]):
                if self.px>0:
                    if stol < self.px: new[st][stol] = array[st][0]
                    else: new[st][stol] = array[st][stol-self.px]
                if self.px<0:
                    if stol >= dim[1]+self.px: new[st][stol] = array[st][dim[1]-1]
                    else: new[st][stol] = array[st][stol-self.px]

        new_tensor = torch.reshape(torch.from_numpy(new).float(), tensor.shape)
        return new_tensor
    
    def __repr__(self):
        return self.__class__.__name__ + '(pixel=)'+str(self.level)

In [ ]:
# Read shot noise and read noise modelling

class Add_Uniform_Noise(object):
    def __init__(self, level = 10):
        self.level = level
        
    def __call__(self, tensor):
        return tensor + (2*self.level*torch.rand(tensor.size())-self.level)
    
    def __repr__(self):
        return self.__class__.__name__ + '(level=)'+str(self.level)

In [ ]:
# Photon noise modelling
class Add_Photon_Noise(object):
    def __init__(self, level = 0.025):
        self.level = level
        
    def __call__(self, tensor):
        return tensor + self.level*torch.sqrt(torch.abs(tensor))*torch.poisson(torch.abs(tensor))
    
    def __repr__(self):
        return self.__class__.__name__ + '(level=)'+str(self.level)

# Prepare cross-validation datasets

In [ ]:
gen_path = 'full_data/'

case_name = '2D_CNN_100_0_001_baseline' # the nameof the experiment
# case_name = model_name + stopping_criterion + learning_rate + comment
cnn_2d_path = case_name+'/'
Y = pd.read_csv(gen_path+'Y_ions.csv', sep=';')


k_folds = [[42,12],[612,45],[72,172],[871,48],[52,134]] #cross-validation folds

'''Write files with labels for each set (training/validation/test) within cross-validation fold - annotation files for CDS_2D_Dataset'''

for fold in k_folds:
  split_path = cnn_2d_path+'split_'+ str(fold[0])+'_'+str(fold[1])+ '/'
  os.makedirs(split_path, exist_ok=True)

  Y_trn, Y_30 = train_test_split(Y, test_size=0.3, random_state=fold[0])
  Y_vld, Y_tst = train_test_split(Y_30, test_size = 0.3333, random_state=fold[1])

  a = ['sample_number','Cu','Ni','Cr','NO3']

  pd.DataFrame(Y_trn).to_csv(split_path + 'Y_trn.csv',sep=',', index=False, header = a)
  pd.DataFrame(Y_vld).to_csv(split_path + 'Y_vld.csv',sep=',', index=False, header = a)
  pd.DataFrame(Y_tst).to_csv(split_path + 'Y_tst.csv',sep=',', index=False, header = a)

In [ ]:
!pip install wandb -qqq
import wandb

     |████████████████████████████████| 1.7 MB 5.1 MB/s 
     |████████████████████████████████| 180 kB 42.7 MB/s 
     |████████████████████████████████| 97 kB 6.9 MB/s 
     |████████████████████████████████| 140 kB 46.7 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 


In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

model = TwoDCNN() # define model

loss_function = torch.nn.MSELoss().cuda() #define loss_function
optimizer = optim.Adam(model.parameters(), lr=0.001)

for fold in k_folds:
    split_path = cnn_2d_path+'split_'+ str(fold[0])+'_'+str(fold[1])+ '/'
    
    training_data = CDS_2D_Dataset(split_path+'Y_trn.csv',gen_path)
    
    validation_data = CDS_2D_Dataset(split_path+'Y_vld.csv',gen_path)
    test_data = CDS_2D_Dataset(split_path+'Y_tst.csv',gen_path)

    train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
    mean, std = mean_std(train_dataloader)

    #Data import and normalization Basic

    training_data = CDS_2D_Dataset(split_path+'Y_trn.csv', gen_path, transform= transforms.Compose([transforms.Normalize(mean, std)]))

    validation_data = CDS_2D_Dataset(split_path+'Y_vld.csv', gen_path, transform= transforms.Compose([transforms.Normalize(mean, std)]))
    test_data = CDS_2D_Dataset(split_path+'Y_tst.csv', gen_path, transform= transforms.Compose([transforms.Normalize(mean, std)]))
    
    train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
    validation_dataloader = DataLoader(validation_data, batch_size=64, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

    #Augmentation via wavlength drifting modelling
    #For data augmentation comment 'Data import and normalization Basic' and use this block instead
    '''
    training_data = CDS_2D_Dataset(split_path+'Y_trn.csv', gen_path, transform= transforms.Compose([transforms.Normalize(mean,std)]))
    training_data_shift_1 = CDS_2D_Dataset(split_path+'Y_trn.csv',gen_path, transform=transforms.Compose([Shift_by_px(1),transforms.Normalize(mean,std)]))
    training_data_shift_2 = CDS_2D_Dataset(split_path+'Y_trn.csv',gen_path, transform=transforms.Compose([Shift_by_px(-1),transforms.Normalize(mean,std)]))
    training_data_concatenated = torch.utils.data.ConcatDataset([training_data, training_data_shift_1, training_data_shift_2])

    validation_data = CDS_2D_Dataset(split_path+'Y_vld.csv', gen_path, transform= transforms.Compose([transforms.Normalize(mean,std)]))
    test_data = CDS_2D_Dataset(split_path+'Y_tst.csv', gen_path, transform= transforms.Compose([transforms.Normalize(mean,std)]))

    train_dataloader = DataLoader(training_data_concatenated, batch_size=64, shuffle=True)
    validation_dataloader = DataLoader(validation_data, batch_size=64, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)
    '''
    
    #Augmentation via uniform noise modelling
    #For data augmentation comment 'Data import and normalization Basic' and use this block instead
    # Noise level parameters: [2.5, 5, 10]
    '''
    training_data = CDS_2D_Dataset(split_path+'Y_trn.csv', gen_path, transform= transforms.Compose([transforms.Normalize(mean,std)]))
    training_data_noise = CDS_2D_Dataset(split_path+'Y_trn.csv',gen_path, transform=transforms.Compose([Add_Uniform_Noise(2.5),transforms.Normalize(mean,std)]))
    training_data_concatenated = torch.utils.data.ConcatDataset([training_data, training_data_noise])

    validation_data = CDS_2D_Dataset(split_path+'Y_vld.csv', gen_path, transform= transforms.Compose([transforms.Normalize(mean,std)]))
    test_data = CDS_2D_Dataset(split_path+'Y_tst.csv', gen_path, transform= transforms.Compose([transforms.Normalize(mean,std)]))

    train_dataloader = DataLoader(training_data_concatenated, batch_size=64, shuffle=True)
    validation_dataloader = DataLoader(validation_data, batch_size=64, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

    '''
    #Augmentation via photon noise modelling
    #For data augmentation comment 'Data import and normalization Basic' and use this block instead
    # Noise level parameters: [0.025, 0.05, 0.1]
    '''
    training_data = CDS_2D_Dataset(split_path+'Y_trn.csv', gen_path, transform= transforms.Compose([transforms.Normalize(mean,std)]))
    training_data_noise = CDS_2D_Dataset(split_path+'Y_trn.csv',gen_path, transform=transforms.Compose([Add_Photon_Noise(0.025),transforms.Normalize(mean,std)]))
    training_data_concatenated = torch.utils.data.ConcatDataset([training_data, training_data_noise])

    validation_data = CDS_2D_Dataset(split_path+'Y_vld.csv', gen_path, transform= transforms.Compose([transforms.Normalize(mean,std)]))
    test_data = CDS_2D_Dataset(split_path+'Y_tst.csv', gen_path, transform= transforms.Compose([transforms.Normalize(mean,std)]))

    train_dataloader = DataLoader(training_data_concatenated, batch_size=64, shuffle=True)
    validation_dataloader = DataLoader(validation_data, batch_size=64, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

    '''

    for init_number in range(0,5): #Это я делаю множественную инициализацию весов сети

        init_path = split_path + str(init_number)+'/'
        os.makedirs(init_path, exist_ok=True)

        # Обучение
        # критерием остановки является ошибка на валидационном наборе - останавливаемся,
        #если в течение 100 эпох (test_stop) ошибка на валидационном наборе (val_loss) не падала

        test_stop = 100 #stopping criterion
        max_val_loss = 10000.0

        wandb.init(project = case_name)

        split_name = 'split_'+ str(fold[0])+'_'+str(fold[1])
        init_name = '_' + str(init_number)
        model_name = '_2D_CNN'
        wandb.run.name = split_name + init_name + model_name
        wandb.run.save()

        for epoch_step in range(0, 1000, test_stop):
            
            if epoch_step!=0:
                checkpoint = torch.load(init_path + 'model'+model_name+'.pth')
                model.load_state_dict(checkpoint['model_state_dict'])
                optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                last_best_epoch = checkpoint['epoch']
                loss = checkpoint['loss']
                model.train()
                
                if last_best_epoch + test_stop > ep:
                    for ep in range(epoch_step, epoch_step+test_stop):
                        for _, data in enumerate(train_dataloader, 0): # get bacth
                            inputs, labels = data # parse batch
                            optimizer.zero_grad() # sets the gradients of all optimized tensors to zero.
                            outputs = model(inputs) # get outputs
                            loss = loss_function(outputs, labels) # calculate loss
                            loss.backward() # calculate gradients
                            optimizer.step() # performs a single optimization step (parameter update).

                        dl = 0
                        val_loss = 0.0
                        for specs, labels in validation_dataloader: 
                            val_loss += loss_function(model(specs),labels)
                            dl+=1
                        val_loss = val_loss/dl
                            
                            
                            
                        if val_loss.item() <= max_val_loss:
                            torch.save({'epoch': ep,
                              'model_state_dict': model.state_dict(),
                              'optimizer_state_dict': optimizer.state_dict(),
                              'loss': loss}, init_path + 'model'+model_name+'.pth')
                            max_val_loss = val_loss.item()
                        wandb.log({"trn_loss": loss, "vld_loss": val_loss})
                else: continue    
      
            if epoch_step==0:
                model.apply(reset_weights)

                for ep in range(epoch_step, test_stop):
                    for _, data in enumerate(train_dataloader, 0): # get bacth
                        inputs, labels = data # parse batch
                        optimizer.zero_grad() # sets the gradients of all optimized tensors to zero.
                        outputs = model(inputs) # get outputs
                        loss = loss_function(outputs, labels) # calculate loss
                        loss.backward() # calculate gradients
                        optimizer.step() # performs a single optimization step (parameter update).

                    dl = 0
                    val_loss = 0.0
                    for specs, labels in validation_dataloader: 
                        val_loss += loss_function(model(specs),labels)
                        dl+=1
                    val_loss = val_loss/dl

                    if val_loss.item() <= max_val_loss:
                        torch.save({'epoch': ep,
                          'model_state_dict': model.state_dict(),
                          'optimizer_state_dict': optimizer.state_dict(),
                          'loss': loss}, init_path + 'model'+model_name+'.pth')
                        max_val_loss = val_loss.item()
                    wandb.log({"trn_loss": loss, "vld_loss": val_loss})

        trn_metrics = calculate_metrics(model, train_dataloader)
        vld_metrics = calculate_metrics(model, validation_dataloader)
        tst_metrics = calculate_metrics(model, test_dataloader)

        wandb.log({"trn_mae": trn_metrics[0].sum()/4, "trn_rmse": trn_metrics[1].sum()/4,"trn_r2": trn_metrics[2].sum()/4})
        wandb.log({"vld_mae": vld_metrics[0].sum()/4, "vld_rmse": vld_metrics[1].sum()/4,"vld_r2": vld_metrics[2].sum()/4})
        wandb.log({"tst_mae": tst_metrics[0].sum()/4, "tst_rmse": tst_metrics[1].sum()/4,"tst_r2": tst_metrics[2].sum()/4})
        wandb.log({"epoch": ep})

        wandb.finish()

        write_predictions(model, model_name, init_path, train_dataloader, dset = 'trn')
        write_predictions(model, model_name, init_path, validation_dataloader, dset ='vld')
        write_predictions(model, model_name, init_path, test_dataloader, dset ='tst')


Using device: cuda


trn_loss,▁
vld_loss,▁
trn_loss,14.86788
vld_loss,12.8185


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Reset trainable parameters of layer = Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
Reset trainable parameters of layer = Linear(in_features=54656, out_features=160, bias=True)
Reset trainable parameters of layer = Linear(in_features=160, out_features=4, bias=True)


epoch,▁
trn_loss,█▅▂▁▂▁▁▁▁▁
trn_mae,▁
trn_r2,▁
trn_rmse,▁
tst_mae,▁
tst_r2,▁
tst_rmse,▁
vld_loss,█▄▂▁▁▁▂▁▁▁
vld_mae,▁
vld_r2,▁


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Reset trainable parameters of layer = Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
Reset trainable parameters of layer = Linear(in_features=54656, out_features=160, bias=True)
Reset trainable parameters of layer = Linear(in_features=160, out_features=4, bias=True)


KeyboardInterrupt: ignored